In [ ]:
# Load Libraries
import pandas as pd
import numpy as np

# Data Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Pre Processing
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Model Selection
from sklearn.model_selection import train_test_split, cross_val_score, KFold, RandomizedSearchCV

# Model
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor

# Metrics
from sklearn.metrics import mean_squared_error, r2_score

# Feature Selection
from sklearn.feature_selection import SelectKBest, chi2

# Warnings 
import warnings as ws
ws.filterwarnings('ignore')

# Save Model
import dill

sns.set_style('whitegrid')
pd.pandas.set_option('display.max_columns',None)

In [ ]:
# Read Dataset
data = pd.read_csv('../input/brasilian-houses-to-rent/houses_to_rent_v2.csv')
data.head()

### Basic Understand about data
    1. Shape
    2. Variables Types
    3. Missing Values
    4. Unique Values

**1. Shape**

In [ ]:
print('Shape of the data : {}'.format(data.shape))

**2. Variables Types**

In [ ]:
data.columns.groupby(data.dtypes)

Floor contains numeric values but this data type is Object.

In [ ]:
data['floor'].value_counts()

In [ ]:
data['floor'] = pd.to_numeric(data['floor'].replace({'-':0}))

Because of '-' this variable convert itself as object. we have to replace with 0

**3. Missing Values** 

In [ ]:
data.isnull().sum()

**4. Unique Values**

In [ ]:
data.nunique()

So we have 13 columns out of which 4 is categorical rest of the variables are numerical. 3 Numerical variables are discreate variables. 

We don't have any missing values in our dataset

### Data Visualization 

**Univariate Analysis**

In [ ]:
data.hist(bins = 25, figsize = (10,10))
plt.show()

All numerical variables are right skewed and few of them are affected by outliers.

In [ ]:
def outlier_detection(feature):
    plt.figure(figsize = (15,3))
    # Box Plot
    plt.subplot(1,3,1)
    sns.boxplot(x = feature,showfliers = False)
    plt.title('Box Plot')
    
    # Distribution Plot
    plt.subplot(1,3,2)
    sns.distplot(feature, kde = False)
    plt.title('Distribution Plot')
    
    # After Log Distriburion plot
    plt.subplot(1,3,3)
    sns.distplot(np.log(feature+1), kde = False)
    plt.title('Distribution Plot after log')
    
    min = feature.min()
    
    result = pd.DataFrame(columns = ['Min','25%','50%','75%','95%','Max'])
    result.loc[0] = [feature.min(),np.quantile(feature,.25),np.quantile(feature,.50),np.quantile(feature,.75),np.quantile(feature,.95), feature.max()]
    
    print(result)


In [ ]:
# Area
outlier_detection(data['area'])

In [ ]:
# hoa (R$)
outlier_detection(data['hoa (R$)'])

In [ ]:
data['hoa (R$)'][data['hoa (R$)'] == 0].count()

This is clearly say that, 2373 values in hoa is 0. After taking log transformation data seems like normally distributed.

In [ ]:
#rent amount
outlier_detection(data['rent amount (R$)'])

In [ ]:
outlier_detection(data['property tax (R$)'])

In [ ]:
data['property tax (R$)'][data['property tax (R$)'] == 0].count()

In [ ]:
outlier_detection(data['total (R$)'])

In [ ]:
# Area
outlier_detection(data['floor'])

**Categorical Variables** 

In [ ]:
['city', 'floor', 'animal', 'furniture']

In [ ]:
# City
sns.countplot(x = 'city', data = data)

In [ ]:
# Floor
sns.countplot(x = 'animal', data = data)

In [ ]:
sns.countplot(x = 'furniture', data = data)

**Bivariate Analysis** 

In [ ]:
sns.pairplot(data)

In [ ]:
def pair_plot(feature, hue = None):
    plt.figure(figsize = (15,3))
    plt.subplot(1,2,1)
    sns.scatterplot(x = data['total (R$)'], y = feature, hue = hue)
    plt.title('Pairplot')
    
    plt.subplot(1,2,2)
    sns.scatterplot( x = np.log(data['total (R$)']), y = np.log(feature+1), hue = hue)
    plt.title('Pairplot after log transform')
    

In [ ]:
pair_plot(data['area'], data['city'])

In [ ]:
pair_plot(data['rent amount (R$)'], data['city'])

In [ ]:
pair_plot(data['hoa (R$)'], data['city'])

In [ ]:
pair_plot(data['property tax (R$)'], data['city'])

In [ ]:
pair_plot(data['fire insurance (R$)'], data['city'])

In [ ]:
pair_plot(data['floor'], data['city'])

In [ ]:
sns.boxplot(x = 'animal', y = 'total (R$)', data = data, showfliers = False)

In [ ]:
sns.boxplot(x = 'furniture', y = 'total (R$)', data = data, showfliers = False)

In [ ]:
sns.boxplot(x = 'city', y = 'total (R$)', data = data, showfliers = False)

In [ ]:
sns.boxplot(x = 'rooms', y = 'total (R$)', data = data, showfliers = False)

In [ ]:
sns.boxplot(x = 'bathroom', y = 'total (R$)', data = data, showfliers = False)

In [ ]:
sns.boxplot(x = 'parking spaces', y = 'total (R$)', data = data, showfliers = False)

#### Corrleation 

In [ ]:
# Correlation Map
corr = data.corr()
mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask)] = True
# Heatmap
plt.figure(figsize=(15, 10))
sns.heatmap(corr, annot = True, fmt = '.2f', mask = mask, linewidths = 2, cmap = 'YlGnBu')
plt.plot()

#### Conclusion From EDA
1. Some Columns are highly affected by outliers. After making log transformation its create linear pattern with target variables.
2. House price is high in São Paulo while compare to other cities.
3. The 'total' is highly correlated with 'hoa', The 'rent amount' is highly correlated with 'fire insurance'
4. If number of Floor, Bathroom, Rooms, Parking Spaces increase total price also will increase.

## PreProcessing
### Encoding
**Label Encoding**

In [ ]:
df['animal'] = df['animal'].replace({'not acept':0, 'acept':1})
df['furniture'] = df['furniture'].replace({'not furnished':0, 'furnished':1})

**One Hot Encoding**

In [ ]:
df['city'] = pd.get_dummies(df['city'], drop_first= True)

## Model Selection 

#### Train Test Split 

In [ ]:
X = df.drop(['total (R$)'], axis = 1)
Y = df['total (R$)']

x_train, x_test,y_train, y_test = train_test_split(X,Y, test_size = 0.3, random_state = 7)

We know that linearly relationship exists between variables. So we can apply linear regression for this dataset.

### Model Validation

Here data seems like linear. So that we can apply linear model on this dataset

In [ ]:
# Models
# Models
models = []
models.append(('LR', LinearRegression()))
models.append(('LASSO', Lasso()))
models.append(('Ridge', Ridge()))
models.append(('EN', ElasticNet()))

In [ ]:
results = []
names = []

col = ['Model', 'MSE Mean','MSE Std']
model_result = pd.DataFrame(columns = col)
i= 0
for name, model in models:
    kfold = KFold(n_splits = 10, random_state = 42)
    cv_results = cross_val_score(model, x_train, y_train, cv = kfold, scoring = 'neg_mean_squared_error')
    
    results.append(cv_results)
    names.append(name)
    
    model_result.loc[i] = [name, cv_results.mean(), cv_results.std()]
    i +=1

    
plt.figure(figsize = (7,4))
sns.boxplot(x = names , y= results, showfliers = False)
plt.tight_layout()
plt.title('MSE')
plt.show()

model_result.sort_values('MSE Mean',ascending = False)
model_result

Here Linear and Ridge model perform well on this dataset 

In [ ]:
lr = LinearRegression()
lr.fit(x_train, y_train)
y_pred = lr.predict(x_test)
mse = mean_squared_error(y_test, y_pred)
print('RMSE Value is : {:.5f}'.format(np.sqrt(mse)))

In [ ]:
y_test_log = np.log(y_test+1)
y_pred_log = np.log(y_pred+1)
sns.regplot(x = y_pred_log, y = y_test_log)

In [ ]:
err = y_test - y_pred
sns.distplot(err)